## Data Scraping from WhoScored
Thank you to [José Ramón Arias González](https://github.com/joseramon-arias/scraper-whoscored) for the inspiration. 

Imports:

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import numpy as np
import time
import bs4
import ast

# 
List of country URLS for getting data from:

In [2]:
country_dict = {'germany': 'https://www.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga', 
                'france': 'https://www.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1',
               'spain': 'https://www.whoscored.com/Regions/206/Tournaments/4/Spain-LaLiga',
                'england': 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/1256/England-Premier-League',
                'italy': 'https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/3054/Italy-Serie-A',
               'turkey': 'https://www.whoscored.com/Regions/225/Tournaments/17/Seasons/5411/Turkey-Super-Lig',
                'netherlands': 'https://www.whoscored.com/Regions/155/Tournaments/13/Netherlands-Eredivisie',
               'portugal': 'https://www.whoscored.com/Regions/177/Tournaments/21/Portugal-Liga-NOS'}

# 
This function iterates through the players historical statistics and returns them as a list of tables:

In [3]:
# iterate through the categories in details tab
def player_table_grabber(category, driver):
    player_table = []
    player_table2 = []
    
    # go to detailed statistics tab
    try:
        element = WebDriverWait(driver.driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='#player-tournament-stats-detailed']")))
        driver.driver.execute_script("arguments[0].click();", element)
        time.sleep(2)
    except Exception as e:

        print(str(e))
        player_table.append('Undefined')
    
    
    # select category
    element = driver.driver.find_element_by_id('category')
    for option in element.find_elements_by_tag_name('option'):
        if option.text == category:
            option.click()
            time.sleep(1.2)
            break

    # special selection for shots because we want the "accuracy" subcategory
    if category == 'Shots':
        element = driver.driver.find_element_by_id('subcategory')
        for option in element.find_elements_by_tag_name('option'):
            if option.text == 'Accuracy':
                option.click()
                time.sleep(1.2)
                break

    content = driver.driver.page_source
    soup = bs4.BeautifulSoup(''.join(content), 'lxml')

    # append the table to the player_table list
    try:
        stats_table = soup.find("div", {"id": "statistics-table-detailed"}).find("tbody", {"id": "player-table-statistics-body"})
        player_table.append(stats_table)
    except Exception as e:

        print(category, count)
        player_table.append('Undefined')

    # special selection for passes because we also want pass type data
    if category == 'Passes':
        
        element = driver.driver.find_element_by_id('subcategory')
        for option in element.find_elements_by_tag_name('option'):
            if option.text == 'Type':
                option.click()
                time.sleep(1.2)
                break

            content = driver.driver.page_source
            soup = bs4.BeautifulSoup(''.join(content), 'lxml')

        # append data to player_table2
            try:
                #Get the table with the data
                stats_table = soup.find("div", {"id": "statistics-table-detailed"}).find("tbody", {"id": "player-table-statistics-body"})
                player_table2.append(stats_table)
            except Exception as e:
                print('exception')
                player_table2.append('Undefined')
        
    # return single table unless passes category, then return both pass tables
    if category != 'Passes':
        return player_table
    else:
        return [player_table, player_table2]


In [4]:
class Country:
    '''class for gathering football data'''
    def __init__(self, name):
        self.name = name.lower()
    def start_driver(self):
        self.driver = webdriver.Chrome()
        self.driver.get(country_dict[self.name])
        self.df = pd.DataFrame()

    def get_data(self, **kwargs):
        if 'end_year' not in kwargs.keys():
            end_year = start_year
        for year in range(kwargs['start_year'],kwargs['end_year']+1):
            
            # select year
            selected_year = self.driver.find_element_by_xpath('//*[@id="seasons"]')
            for option in selected_year.find_elements_by_tag_name('option'):
                if option.text == str(year-1) + '/' + str(year):
                    option.click()
                    time.sleep(2)
                    break

            if self.name == 'netherlands':
                element = self.driver.find_element_by_id('stages')
                for option in element.find_elements_by_tag_name('option'):
                    if option.text == 'Eredivisie':
                        option.click()
                        time.sleep(2)
                        break
            
            # navigate to player statistics
            player_statistics_link = self.driver.find_element_by_xpath('//*[@id="sub-navigation"]/ul/li[4]/a')
            player_statistics_link.click()
            print('Getting data for ' + self.name + ' (' + str(year) + ')')
            time.sleep(4)  

            # set to all players rather than based on appearances
            all_players = self.driver.find_element_by_xpath('//*[@id="apps"]/dd[2]/a')
            all_players.click()
            time.sleep(2) 
            
            # click on the detailed tab of the table
            try:
                element = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='#stage-top-player-stats-detailed']")))
                self.driver.execute_script("arguments[0].click();", element)
                time.sleep(2)
            except Exception as e:

                print(str(e))
                player_tables.append('Undefined')


            # age selection
            
            if 'max_age' in kwargs.keys():
                element = self.driver.find_element_by_id('ageComparisonType')
                for option in element.find_elements_by_tag_name('option'):
                    if option.text == 'Less than':
                        option.click()
                        time.sleep(1)

            age_input = self.driver.find_element_by_id('age')
        
            age_input.send_keys(2021 - year + kwargs['max_age'])
            time.sleep(2)


            # position choice ('attack', 'midfield', 'defense', 'gk')
            
            if 'position' in kwargs.keys():
                element = self.driver.find_element_by_id('position')
                element.click()
                time.sleep(1)
                element = self.driver.find_element_by_id('toggle')
                element.click()
                time.sleep(1)

                for position_choice in kwargs['position']:
                    if position_choice == 'attack':
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[1]/td[1]/label/input')
                        element.click()
                        time.sleep(.1)
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[2]/td[1]/label/input')
                        element.click()
                        time.sleep(.1)
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[2]/td[2]/label/input')
                        element.click()
                        time.sleep(.1)
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[2]/td[3]/label/input')
                        element.click()
                        time.sleep(.1)
                    if position_choice == 'midfield':
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[3]/td[1]/label/input')
                        element.click()
                        time.sleep(.1)
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[3]/td[2]/label/input')
                        element.click()
                        time.sleep(.1)
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[3]/td[3]/label/input')
                        element.click()
                        time.sleep(.1)
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[4]/td/label/input')
                        element.click()
                        time.sleep(.1)
                    if position_choice == 'defense':
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[5]/td[1]/label/input')
                        element.click()
                        time.sleep(.1)
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[5]/td[2]/label/input')
                        element.click()
                        time.sleep(.1)
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[5]/td[3]/label/input')
                        element.click()
                        time.sleep(.1)
                    if position_choice == 'gk':
                        element = self.driver.find_element_by_xpath('//*[@id="pitch"]/tbody/tr[6]/td/label/input')
                        element.click()
                        time.sleep(.1)

            # set minimum appearances (defaults to 6)
            if 'apps' in kwargs.keys():
                element = self.driver.find_element_by_id('appearancesComparisonType')
                for option in element.find_elements_by_tag_name('option'):
                    if option.text == 'More than':
                        option.click()
                        time.sleep(1)
                input_ = self.driver.find_element_by_id('appearances')
                input_.send_keys(kwargs['apps']-1)
                time.sleep(1) 

            # apply filters
            element = self.driver.find_element_by_class_name('search-button')
            while True:
                try:
                    element.click()
                    time.sleep(2)
                except:
                    continue
                break
            
            players = []

            # get list of applicable players

            # grab data and click next page until the last page is reached
            while int(self.driver.find_elements_by_tag_name('b')[1].text.split()[-3]) != int(self.driver.find_elements_by_tag_name('b')[1].text.split()[-1]): # these will be equal when on lat page
                content = self.driver.page_source
                soup = bs4.BeautifulSoup(''.join(content), 'lxml')
                next_page = self.driver.find_element_by_xpath('//*[@id="statistics-paging-detailed"]/div/dl[2]/dd[3]')
                try:
                    next_page.click()
                except:
                    next_page = self.driver.find_element_by_xpath('//*[@id="statistics-paging-detailed"]/div/dl[2]/dd[3]')
                    next_page.click()
                time.sleep(2)


                 #Get the table with the data
                stats_table = soup.find("div", {"id": "statistics-table-detailed"}).find("tbody", {"id": "player-table-statistics-body"})
                players.append(stats_table)

            # grab data for last page
            content = self.driver.page_source
            soup = bs4.BeautifulSoup(''.join(content), 'lxml')
            #Get the table with the data
            stats_table = soup.find("div", {"id": "statistics-table-detailed"}).find("tbody", {"id": "player-table-statistics-body"})
            players.append(stats_table)
            
            # get player links
        
        player_links = []
        for table in players:
            ages = table.find_all('span', {'class': 'player-meta-data'})
            ages = ages[0::2]
            ages_list = []
            players_list = []
            for age in ages:
                ages_list.append(int(age.text))
            players2 = table.find_all('a', {'class':'player-link'})
            players2 = players2[0::2]
            for player_link in players2:
                players_list.append(player_link['href'])
            player_links.append([(p1, p2) for idx1, p1 in enumerate(ages_list) for idx2, p2 in enumerate(players_list) if idx1 == idx2])
        print(player_links)
        player_tables = []
        
        # iterate through player links and get player data tables
        
        for links in player_links:
            for pair in links:
                while True:
                    try:
                        player_link = pair[1]
                        name = player_link[player_link.find('/Show')+ 6:].replace('-', ' ')
                        player_df = pd.DataFrame(columns = ['name', 'age', 'season', 'tournament', 'club', 'apps', 'mins'])
                        age = pair[0]
                       
                        self.driver.get('https://www.whoscored.com' + player_link.replace('/Show', '/History'))
                        time.sleep(1)
                        
                            
                        # get table for each category for each player
                        if 'category' not in kwargs.keys():
                            kwargs['category'] = ['Shots', 'Goals', 'Dribbles', 'Possession loss', 'Aerial', 'Passes', 'Key passes', 'Assists']
                       
                        for category in kwargs['category']:
                            globals()[category.replace(' ', '_').lower()] = player_table_grabber(category, self)
                        
                        trs = globals()[kwargs['category'][0].replace(' ', '_').lower()][0].find_all('tr')
                        for tr in trs[:-1]:

                            undefined_table = ['undefined']*7

                            tds = tr.find_all('td')
                            club_href = tds[1].find('a')

                            club_href = club_href['href']

                            club = club_href.split('/Show')[1].split('-', 1)[1].replace('-', ' ').replace(' ', '')
                            season = tds[0].text.split('/')[-1]
                            name = name

                            age_ = age - (2021-int(season))
                            tournament = tds[4].text
                            apps = float(tds[5].get_text().strip())
                            mins = float(tds[6].get_text().strip())


                            undefined_table[0] = name if name != '-' else 'undefined'
                            undefined_table[1] = float(age_) if age_ != '-' else 'undefined'
                            undefined_table[2] = season if season != '-' else 'undefined'
                            undefined_table[3] = tournament if tournament != '-' else 'undefined'
                            undefined_table[4] = club if club != '-' else 'undefined'  
                            undefined_table[5] = apps if apps != '-' else 'undefined'
                            undefined_table[6] = mins if mins != '-' else 'undefined'

                            df = pd.Series(undefined_table, index = ['name', 'age', 'season', 'tournament', 'club', 'apps', 'mins']) #'shots (off target)', 'shots (on target)', 'shots (blocked)'])        
                            player_df = player_df.append(df, ignore_index = True)
                
                        for category in kwargs['category']:
                            if category == 'Shots':
                                # goals data

                                trs = shots[0].find_all('tr')
                                player_data = []
                                off_target_list = []
                                on_target_list = []
                                blocked_list = []
                                for tr in trs[:-1]:



                                    tds = tr.find_all('td')

                                    off_target = tds[8].get_text().strip()
                                    on_target = tds[10].get_text().strip()
                                    blocked = tds[11].get_text().strip()

                                    off_target_list.append(float(off_target)) if off_target != '-' else off_target_list.append('undefined')
                                    on_target_list.append(float(on_target)) if on_target != '-' else on_target_list.append('undefined')
                                    blocked_list.append(float(blocked)) if blocked != '-' else blocked_list.append('undefined')



                                player_df = player_df.assign(shots_off_target = off_target_list)
                                player_df = player_df.assign(shots_on_target = on_target_list)
                                player_df = player_df.assign(shots_blocked = blocked_list)
                                




                            if category == 'Goals':

            
                                # goals data

                                trs = goals[0].find_all('tr')
                                player_data = []
                                goals_list = []
                                outside_box_list = []
                                for tr in trs[:-1]:



                                    tds = tr.find_all('td')

                                    total_goals = tds[7].text.replace('\t', '')
                                    outside_of_box = tds[10].text.replace('\t', '')

                                    goals_list.append(float(total_goals)) if total_goals != '-' else goals_list.append('undefined')
                                    outside_box_list.append(float(outside_of_box)) if outside_of_box != '-' else outside_box_list.append('undefined')



                                player_df = player_df.assign(goals = goals_list)
                                player_df = player_df.assign(outside_of_box_goals = outside_box_list)
                                
                            if category == 'Dribbles':
                                

                                # dribbles data

                                trs = dribbles[0].find_all('tr')
                                u_dribbles_list = []
                                s_dribbles_list = []
                                for tr in trs[:-1]:


                                    tds = tr.find_all('td')

                                    unsuccessful_dribbles = tds[7].get_text().strip()
                                    successful_dribbles = tds[8].get_text().strip()

                                    u_dribbles_list.append(float(unsuccessful_dribbles)) if unsuccessful_dribbles != '-' else u_dribbles_list.append('undefined')
                                    s_dribbles_list.append(float(successful_dribbles)) if successful_dribbles != '-' else s_dribbles_list.append('undefined')


                                player_df = player_df.assign(unsuccessful_dribbles = u_dribbles_list)
                                player_df = player_df.assign(successful_dribbles = s_dribbles_list)
                                

                                # possession loss data
                            if category == 'Possession loss':
                                trs = possession_loss[0].find_all('tr')
                                u_touches_list = []
                                dispossessed_list = []
                                for tr in trs[:-1]:


                                    tds = tr.find_all('td')

                                    unsuccessful_touches = tds[7].get_text().strip().replace(' ', '')
                                    dispossessed = tds[8].get_text().strip().replace(' ', '')

                                    u_touches_list.append(float(unsuccessful_touches)) if unsuccessful_touches != '-' else u_touches_list.append('undefined')
                                    dispossessed_list.append(float(dispossessed)) if dispossessed != '-' else dispossessed_list.append('undefined')


                                player_df = player_df.assign(unsuccesful_touches = u_touches_list)
                                player_df = player_df.assign(dispossessed = dispossessed_list)
                                


                                # aerial data
                            if category == 'Aerial':
                                trs = aerial[0].find_all('tr')
                                aw_list = []
                                al_list = []
                                for tr in trs[:-1]:


                                    tds = tr.find_all('td')

                                    aerial_won = tds[8].get_text().strip().replace(' ', '')
                                    aerial_lost = tds[9].get_text().strip().replace(' ', '')

                                    aw_list.append(float(aerial_won)) if aerial_won != '-' else aw_list.append('undefined')
                                    al_list.append(float(aerial_lost)) if aerial_lost != '-' else al_list.append('undefined')


                                player_df = player_df.assign(aerial_won = aw_list)
                                player_df = player_df.assign(aerial_lost = al_list)
                                

                                # pass length data
                            if category == 'Passes':
                                trs = passes[0][0].find_all('tr')
                                total_passes_list = []
                                accurate_long_pass_list = []
                                inaccurate_long_pass_list = []
                                accurate_short_pass_list = []
                                inaccurate_short_pass_list = []
                                for tr in trs[:-1]:


                                    tds = tr.find_all('td')

                                    total_passes = tds[7].get_text().strip().replace(' ', '')
                                    accurate_long_pass = tds[8].get_text().strip().replace(' ', '')
                                    inaccurate_long_pass = tds[9].get_text().strip().replace(' ', '')
                                    accurate_short_pass = tds[10].get_text().strip().replace(' ', '')
                                    inaccurate_short_pass = tds[11].get_text().strip().replace(' ', '')




                                    total_passes_list.append(float(total_passes)) if total_passes != '-' else total_passes_list.append('undefined')
                                    accurate_long_pass_list.append(float(accurate_long_pass)) if accurate_long_pass != '-' else accurate_long_pass_list.append('undefined')
                                    inaccurate_long_pass_list.append(float(inaccurate_long_pass)) if inaccurate_long_pass != '-' else inaccurate_long_pass_list.append('undefined')
                                    accurate_short_pass_list.append(float(accurate_short_pass)) if accurate_short_pass != '-' else accurate_short_pass_list.append('undefined')
                                    inaccurate_short_pass_list.append(float(inaccurate_short_pass)) if inaccurate_short_pass != '-' else inaccurate_short_pass_list.append('undefined')


                                player_df = player_df.assign(total_passes = total_passes_list)
                                player_df = player_df.assign(accurate_long_pass = accurate_long_pass_list)
                                player_df = player_df.assign(inaccurate_long_pass = inaccurate_long_pass_list)
                                player_df = player_df.assign(accurate_short_pass = accurate_short_pass_list)
                                player_df = player_df.assign(inaccurate_short_pass = inaccurate_short_pass_list)

                                

                                # pass type data

                                trs = passes[1][0].find_all('tr')
                                accurate_crosses_list = []
                                inaccurate_crosses_list = []

                                for tr in trs[:-1]:

                                    tds = tr.find_all('td')

                                    accurate_crosses = tds[7].get_text().strip().replace(' ', '')
                                    inaccurate_crosses = tds[8].get_text().strip().replace(' ', '')

                                    accurate_crosses_list.append(float(accurate_crosses)) if accurate_crosses != '-' else accurate_crosses_list.append('undefined')
                                    inaccurate_crosses_list.append(float(inaccurate_crosses)) if inaccurate_crosses != '-' else inaccurate_crosses_list.append('undefined')

                                player_df = player_df.assign(accurate_crosses = accurate_crosses_list)
                                player_df = player_df.assign(inaccurate_crosses = inaccurate_crosses_list)
                                

                                # key pass data
                            if category == 'Key passes':
                                trs = key_passes[0].find_all('tr')
                                long_key_pass_list = []
                                short_key_pass_list= []

                                for tr in trs[:-1]:

                                    tds = tr.find_all('td')

                                    long_key_pass = tds[8].get_text().strip().replace(' ', '')
                                    short_key_pass = tds[9].get_text().strip().replace(' ', '')

                                    long_key_pass_list.append(float(long_key_pass)) if long_key_pass != '-' else long_key_pass_list.append('undefined')
                                    short_key_pass_list.append(float(short_key_pass)) if short_key_pass != '-' else short_key_pass_list.append('undefined')

                                player_df = player_df.assign(long_key_pass = long_key_pass_list)
                                player_df = player_df.assign(short_key_pass = short_key_pass_list)

                                
                                # assists data
                            if category == 'Assists':
                                trs = assists[0].find_all('tr')
                                assists_list = []

                                for tr in trs[:-1]:

                                    tds = tr.find_all('td')

                                    assists_ = tds[13].get_text().strip().replace(' ', '')

                                    assists_list.append(float(assists_)) if assists_ != '-' else assists_list.append('undefined')

                                player_df = player_df.assign(assists = assists_list)
                                
                                
                        self.df = pd.concat([self.df, player_df])
                        display(self.df.tail(1))
                    except Exception as e:
                        print(e)
                        continue
                    break



In [5]:
germany = Country('Germany')

In [6]:
germany.name

'germany'

In [7]:
germany.start_driver()

In [8]:
germany.get_data(start_year = 2016, end_year = 2016, category = ['Shots','Passes'], position = ['midfield'], max_age = 22, apps = 10)

Getting data for germany (2016)
[[(25, '/Players/125883/Show/Kingsley-Coman'), (25, '/Players/135092/Show/Mahmoud-Dahoud'), (25, '/Players/115519/Show/Leon-Goretzka'), (25, '/Players/142318/Show/Julian-Weigl'), (26, '/Players/116466/Show/Marc-Stendera'), (26, '/Players/101859/Show/Pierre-Emile-Højbjerg'), (25, '/Players/291937/Show/Florian-Grillitsch'), (25, '/Players/144769/Show/Max-Christiansen'), (25, '/Players/291676/Show/Gideon-Jung'), (25, '/Players/124598/Show/Max-Meyer')], [(25, '/Players/134756/Show/Levin-Öztunali')]]


,name,age,season,tournament,club,apps,mins,shots_off_target,shots_on_target,shots_blocked,total_passes,accurate_long_pass,inaccurate_long_pass,accurate_short_pass,inaccurate_short_pass,accurate_crosses,inaccurate_crosses
19,Kingsley Coman,17.0,2013,FL1,ParisSaintGermain,1.0,3.0,undefined,1.0,undefined,1.0,undefined,undefined,1.0,undefined,1.0,undefined


,name,age,season,tournament,club,apps,mins,shots_off_target,shots_on_target,shots_blocked,total_passes,accurate_long_pass,inaccurate_long_pass,accurate_short_pass,inaccurate_short_pass,accurate_crosses,inaccurate_crosses
16,Mahmoud Dahoud,19.0,2015,UEL,BorussiaMGladbach,1.0,20.0,undefined,undefined,undefined,22.0,undefined,undefined,22.0,undefined,22.0,undefined


,name,age,season,tournament,club,apps,mins,shots_off_target,shots_on_target,shots_blocked,total_passes,accurate_long_pass,inaccurate_long_pass,accurate_short_pass,inaccurate_short_pass,accurate_crosses,inaccurate_crosses
18,Leon Goretzka,18.0,2014,GB,Schalke04,25.0,1503.0,0.7,0.6,0.4,27.1,1.4,0.3,19.4,6.0,27.1,1.4


,name,age,season,tournament,club,apps,mins,shots_off_target,shots_on_target,shots_blocked,total_passes,accurate_long_pass,inaccurate_long_pass,accurate_short_pass,inaccurate_short_pass,accurate_crosses,inaccurate_crosses
14,Julian Weigl,20.0,2016,UEL,BorussiaDortmund,12.0,945.0,0.1,0.3,0.3,76.6,3.8,2.3,65.8,4.7,76.6,3.8


...

,name,age,season,tournament,club,apps,mins,shots_off_target,shots_on_target,shots_blocked,total_passes,accurate_long_pass,inaccurate_long_pass,accurate_short_pass,inaccurate_short_pass,accurate_crosses,inaccurate_crosses
11,Florian Grillitsch,20.0,2016,GB,WerderBremen,25.0,1989.0,0.4,0.3,0.4,35.0,3.0,2.6,23.2,6.2,35.0,3.0


,name,age,season,tournament,club,apps,mins,shots_off_target,shots_on_target,shots_blocked,total_passes,accurate_long_pass,inaccurate_long_pass,accurate_short_pass,inaccurate_short_pass,accurate_crosses,inaccurate_crosses
3,Max Christiansen,20.0,2016,GB,Ingolstadt,19.0,1031.0,0.3,0.2,0.4,22.4,1.0,0.9,14.9,5.6,22.4,1.0


,name,age,season,tournament,club,apps,mins,shots_off_target,shots_on_target,shots_blocked,total_passes,accurate_long_pass,inaccurate_long_pass,accurate_short_pass,inaccurate_short_pass,accurate_crosses,inaccurate_crosses
5,Gideon Jung,20.0,2016,GB,HamburgerSV,19.0,1177.0,0.2,0.2,0.1,24.4,1.5,1.5,17.2,4.3,24.4,1.5


,name,age,season,tournament,club,apps,mins,shots_off_target,shots_on_target,shots_blocked,total_passes,accurate_long_pass,inaccurate_long_pass,accurate_short_pass,inaccurate_short_pass,accurate_crosses,inaccurate_crosses
14,Max Meyer,17.0,2013,UCL,Schalke04,1.0,5.0,undefined,undefined,undefined,5.0,1.0,undefined,3.0,1.0,5.0,1.0


,name,age,season,tournament,club,apps,mins,shots_off_target,shots_on_target,shots_blocked,total_passes,accurate_long_pass,inaccurate_long_pass,accurate_short_pass,inaccurate_short_pass,accurate_crosses,inaccurate_crosses
11,Levin Öztunali,18.0,2014,GB,BayerLeverkusen,9.0,86.0,undefined,undefined,undefined,4.4,0.6,undefined,3.2,0.7,4.4,0.6


In [9]:
germany.df

,name,age,season,tournament,club,apps,mins,shots_off_target,shots_on_target,shots_blocked,total_passes,accurate_long_pass,inaccurate_long_pass,accurate_short_pass,inaccurate_short_pass,accurate_crosses,inaccurate_crosses
0,Kingsley Coman,26.0,2022,WC,France,3.0,123.0,0.7,0.7,undefined,22.3,undefined,0.7,19.0,2.7,22.3,undefined
1,Kingsley Coman,25.0,2021,GB,BayernMunich,29.0,1767.0,0.7,0.7,0.5,25.5,0.9,0.3,20.8,3.6,25.5,0.9
2,Kingsley Coman,25.0,2021,UCL,BayernMunich,7.0,549.0,0.9,0.7,0.9,31.1,1.0,0.1,24.7,5.3,31.1,1.0
3,Kingsley Coman,25.0,2021,UNL,France,4.0,98.0,undefined,0.5,undefined,9.5,0.3,undefined,6.8,2.5,9.5,0.3
4,Kingsley Coman,24.0,2020,GB,BayernMunich,24.0,1507.0,0.7,0.6,0.2,29.4,1.0,0.5,23.2,4.8,29.4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,Levin Öztunali,20.0,2016,GB,WerderBremen,25.0,1410.0,0.5,0.2,0.2,13.0,0.2,0.2,8.2,4.3,13.0,0.2
8,Levin Öztunali,19.0,2015,GB,BayerLeverkusen,6.0,149.0,0.5,undefined,0.5,7.8,undefined,0.3,4.7,2.8,7.8,undefined
9,Levin Öztunali,19.0,2015,GB,WerderBremen,16.0,748.0,0.1,0.4,0.2,12.1,0.4,0.6,7.3,3.8,12.1,0.4
10,Levin Öztunali,19.0,2015,UCL,BayerLeverkusen,1.0,20.0,undefined,undefined,undefined,4.0,1.0,undefined,1.0,2.0,4.0,1.0
